## Missing Persons DB Webscrape

https://www.services.rcmp-grc.gc.ca/missing-disparus/search-recherche.jsf

Search with no criteria will bring up all results.

The links to the missing persons pages, can be appended to https://www.services.rcmp-grc.gc.ca/

### ========== Code

#### Get all the links to the more detailed case pages from the navigation pages

In [12]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
import requests
import json

In [3]:
link = 'https://www.services.rcmp-grc.gc.ca/missing-disparus/search-recherche.jsf'
driver=r'C:\Users\Neptune\Downloads\chromedriver'

# using the selenium web driver I downloaded
browser = webdriver.Chrome(driver)
browser.get(link)
time.sleep(2)

# find search and click it to reach data
try:
    search = browser.find_element_by_name('searchForm:j_idt158')
    time.sleep(2)
except:
    print('could not find Search')
    
try:
    search.click()
except:
    print('could not find Search')

time.sleep(5)

not_last = True
# to store all the URLs
URLs = []

# go through each page and get all URLs    
while not_last:
    time.sleep(2) # wait more just in case
    page = browser.page_source
    pageSoup = bs(page, 'html.parser')
    
    # get all the links on the page and add them to array
    pageURLs = pageSoup.find_all('a',class_="wet-newwindow")
    
    print("Collecting URLs...")
    
    # take each link on the page and add if not a dupe
    for link in pageURLs:
        href = link.get('href')
        if href not in URLs:
            URLs.append(href)

    print("Page Complete!")
    time.sleep(2) # wait a little
    
    # are we on the last page
    try:
        # click the next button at the bottom of the page
        next_page = browser.find_element_by_xpath('/html/body/main/form/div[33]/ul/li[83]/a')
        print('Found next button to press.')
        next_page.click()
        time.sleep(4) # wait for next page to load
    except:
        # should not have a next button on the last page
        print('last page or no next button found!')
        not_last = False

# the final list
print("================================== END ==================================")
print(len(URLs[:30])) # show a section       
time.sleep(5)

# write progress to csv
df = pd.DataFrame(URLs, columns=["URL"])
df.to_csv('list.csv', index=False)

browser.quit()

C:\Users\Neptune\AppData\Local\Temp/ipykernel_926760/387393780.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(driver)
C:\Users\Neptune\AppData\Local\Temp/ipykernel_926760/387393780.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = browser.find_element_by_name('searchForm:j_idt158')


Page Complete!


C:\Users\Neptune\AppData\Local\Temp/ipykernel_926760/387393780.py:50: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_page = browser.find_element_by_xpath('/html/body/main/form/div[33]/ul/li[83]/a')


Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next button to press.
Page Complete!
Found next 

#### Collect all the data from all the detailed case pages

This part of the code adapted from the CBC script

In [9]:
base_url = r'https://www.services.rcmp-grc.gc.ca'

#CLEANING FUNCTION
def cleaning_function(item):
    item = str(item)
    item = item.replace("<dd>" , "")
    item = item.replace("</dd>" , "")
    item = item.replace("<p>" , "")
    item = item.replace("</p>" , "")
    item = item.replace("<strong>Missing from </strong>" , "")
    item = item.replace("<strong>" , "")
    item = item.replace("</strong>" , "")
    return item

In [10]:
#this is the list where all the URLs from the sheet will go
person_url_list = []
#this is where all the person info will go
person_info = []

#a list for the sections later
section_list = []

#I have this because I dont know how else to filter out stuff from an if statement that I dont want
count_working = 0

for page_url in URLs:
    print("Record Number: " + str(count_working))
    print("Case URL: " + page_url)
    count_working += 1
    url = base_url + page_url
    
    # request the html
    try:
        page = requests.get(url, timeout = 10)
    except requests.exceptions.Timeout:
        print("Timeout occurred")
    # structure the page content for parsing
    soup = bs(page.content, 'html.parser') 
    
    # First we have to pull out the content area
    content_area = soup.find('main' , {"property" : "mainContentOfPage"})
    
    # LOCATION
    try:
        location_scrape = content_area.find('div')
        location_isolate = location_scrape.find_all('p')
        location_string = str(location_isolate[2])
        location_split = location_string.split(",")
        province = cleaning_function(location_split[1])
        city = cleaning_function(location_split[0])
    except:
        print('No Location')
    
    # STATUS
    status_scrape = content_area.find_all('h2')
    status = status_scrape[:1]
    status = str(status)
    front_of_status = status.index('<h2>') + 4
    back_of_status = status.index('</h2>')
    status_cleaned = (f'{status[front_of_status : back_of_status]}')
    
    # FOR THE MISSING ENTRIES
    if 'Missing' in status_cleaned:
        #Now we get into pulling out individual details which will eventually be compiled in a list
        #NAME(MISSING)
        name_scrape = content_area.find_all('h3')
        person_name = name_scrape[:1]
        person_name = str(person_name)
        front_of_name = person_name.index('<h3>') + 4
        back_of_name = person_name.index('</h3>')
        name_cleaned = (f'{person_name[front_of_name : back_of_name]}')
        name_split = name_cleaned.split(',')
        last_name = name_split[0]
        first_name = name_split[1]
        first_name_string = str(name_split[1:2])
        first_name_string = first_name_string.replace('[',"")
        first_name_string = first_name_string.replace(']',"")
        first_name_string = first_name_string.replace("'","")
        first_name_string = first_name_string.replace("\n","")
        first_name_string = first_name_string.strip()
        
        #PERSON DETAILS(MISSING)
        try:
            person_details = content_area.find_all('dd')
            date_missing_discovered = person_details[0]
            year_born = person_details[1]
            age_at_disappearance = person_details[2]
            gender = person_details[3]
            bio_group = person_details[4]
        except:
            print('Data error')

    #FOR THE UNIDENTIFIED ENTRIES
    else:
        try:
            first_name_string = 'Unidentified'
            last_name = 'Unidentified'
            person_details = content_area.find_all('dd')
            date_missing_discovered = person_details[0]
            age_at_disappearance = person_details[1]
            gender = person_details[2]
            bio_group = person_details[3]
            year_born = 'Unknown'
        except:
            print('Data error2')
            
            
    #PUT IT ALL TOGETHER
    person_info.append([first_name_string , last_name , status_cleaned , cleaning_function(date_missing_discovered) , cleaning_function(year_born) , cleaning_function(age_at_disappearance) , cleaning_function(gender) , cleaning_function(bio_group) , city , province , url])
       

Record Number: 0
Case URL: /missing-disparus/case-dossier.jsf?case=2019019954&id=0
Record Number: 1
Case URL: /missing-disparus/case-dossier.jsf?case=2014001708&id=1
Record Number: 2
Case URL: /missing-disparus/case-dossier.jsf?case=2014005078&id=2
Record Number: 3
Case URL: /missing-disparus/case-dossier.jsf?case=2014005330&id=3
No Location
Record Number: 4
Case URL: /missing-disparus/case-dossier.jsf?case=2018062818&id=4
Record Number: 5
Case URL: /missing-disparus/case-dossier.jsf?case=2018053584&id=5
Record Number: 6
Case URL: /missing-disparus/case-dossier.jsf?case=2014005453&id=6
Record Number: 7
Case URL: /missing-disparus/case-dossier.jsf?case=2001008160&id=7
Record Number: 8
Case URL: /missing-disparus/case-dossier.jsf?case=2014003507&id=8
Record Number: 9
Case URL: /missing-disparus/case-dossier.jsf?case=2019056482&id=9
Record Number: 10
Case URL: /missing-disparus/case-dossier.jsf?case=2014006470&id=10
Record Number: 11
Case URL: /missing-disparus/case-dossier.jsf?case=20190

NameError: name 'pandas' is not defined

In [11]:
# print to a file
full_file = pd.DataFrame(person_info)
full_file.to_csv("output_rcmp.csv")
print('Done')

Done


### Second Method - For More Structured Data

In [52]:
#I have this because I dont know how else to filter out stuff from an if statement that I dont want
count_working = 0

# complete file
complete_db = {}

test_URLs = URLs[:2]

# loop through all the URLs
for page_url in test_URLs:
    
    # page dict
    page_dict = {}
    #this is where all the person info will go
    page_sections = []
    # make the full URL
    url = base_url + page_url
    
    print('==============================================')
    print("Record Number: " + str(count_working))
    print("Case URL: " + url)
    
    # next record
    count_working += 1
    
    # request the html
    try:
        page = requests.get(url, timeout = 10)
    except requests.exceptions.Timeout:
        print("Timeout occurred")
    
    # structure the page content for parsing
    soup = bs(page.content, 'html.parser') 
    
    # First we have to pull out the content area
    content_area = soup.find('main' , {"property" : "mainContentOfPage"})
    
    try:
        # the case reference number
        _case_ref = content_area.find('h1')
        page_dict['CaseRef'] = " ".join(_case_ref.text.split())
        
        # the main section
        sections = content_area.section
        
        # the description
        desc = sections.div.p
        page_dict['CaseDesc'] = desc.text.strip()
        
        # the category
        case_type = sections.h2
        page_dict['CaseType'] = " ".join(case_type.text.split())
    except:
        print('page base info collection error')
    
    # different types are structured differently
    if page_dict['CaseType'] == 'Missing':
    
        # get the first section with all the persons
        persons_section = sections.section
        
        # how many people are we looking through
        persons_names = persons_section.find_all('h3')
        num_persons = len(persons_names)
        # all the blocks within the section
        persons_blocks = persons_section.find_all('section')
        
        for i in range(num_persons):
            block = {} # stores the individuals info, some pages have 1+
            block['Name'] = " ".join(persons_names[i].text.split())
            
            #print(persons_blocks[i])
            print('-------')
            
            page_sections.append(block)
    
    else:
        print("unid'd")
    
    
    page_dict['PersonsData'] = page_sections
    
    
    print(page_dict)
    
    
print('======================= Done =======================')


Record Number: 0
Case URL: https://www.services.rcmp-grc.gc.ca/missing-disparus/case-dossier.jsf?case=2019019954&id=0
-------
-------
-------
{'CaseRef': 'Case reference: 2019019954', 'CaseDesc': 'On April 14, 2019, at approximately 4:00 p.m., Yonis and Mawahib Ahmed were seen for the last time with their mother Liyana Abdow at the Sojourn House in Toronto, Ontario.', 'CaseType': 'Missing', 'PersonsData': [{'Name': 'AHMED, Yonis'}, {'Name': 'AHMED, Mawahib'}, {'Name': 'ABDOW, Liyana Mohamed'}]}
Record Number: 1
Case URL: https://www.services.rcmp-grc.gc.ca/missing-disparus/case-dossier.jsf?case=2014001708&id=1
-------
{'CaseRef': 'Case reference: 2014001708', 'CaseDesc': 'Marlene Abigosis was last seen in March of 1984 in the downtown east side of Vancouver. Ms. Abigosis was engaged in a high risk lifestyle. M. Abigosis was known to work near the docks, and clients that owned boats would take Marlene with them on trips to Washington State and Vancouver Island on occasion.', 'CaseType':

In [ ]:
# write JSON to a file    
with open("sample.json", "w") as outfile:
    json.dump(complete_db, outfile)